## Imports

In [1]:
%load_ext autoreload
%autoreload

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import sklearn.metrics as metrics
import pandas as pd
import numpy as np
from torch_geometric.data import Data
import pandapower as pp
import networkx as nx
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torch_geometric.nn.norm import BatchNorm
from torch.utils.tensorboard import SummaryWriter
from arquitecturas import GNN_Local
from metric import NormalizedError

/home/iboero/miniconda3/envs/proy/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/iboero/miniconda3/envs/proy/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/tmp/ipykernel_28371/3113086596.py:10: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  im

In [16]:
if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'


red = "118" # o 118

## Levantar Topologia de Red

In [17]:
# Levantamos la red de IEEE118

if red=="30":
    net = pp.networks.case30()
    z_trafos = net.trafo[['hv_bus', 'lv_bus']].to_numpy().astype(np.int32)
else:
    net = pp.networks.case118()
    z_trafos =  np.array([
        [0, 0.0267],
        [0, 0.0382],
        [0, 0.0388],
        [0, 0.0375],
        [0, 0.0386],
        [0, 0.0268],
        [0, 0.0370],
        [0.0013, 0.016],
        [0, 0.0370],
        [0.0017, 0.0202],
        [0, 0.0370],
        [0.0282, 0.2074],
        [0.0003, 0.00405]
    ])

# Armar matirz de adyacencia
lineas = net.line[['from_bus', 'to_bus']].to_numpy().astype(np.int32)
trafos = net.trafo[['hv_bus', 'lv_bus']].to_numpy().astype(np.int32)
edge_index = np.append(lineas, trafos,axis=0)
edge_index = torch.Tensor(edge_index).t().type(torch.int64).to(device)


# Armar matriz de pesos
k = 10
z_lineas =  net.line[['r_ohm_per_km', 'x_ohm_per_km']].to_numpy() * 100 / np.expand_dims((net.bus['vn_kv'].to_numpy()[net.line['from_bus'].to_numpy()])**2,axis=1)

edge_weights = np.append(z_lineas, z_trafos,axis=0)
edge_weights = torch.Tensor(np.e**(-k*(edge_weights[:,0]**2 + edge_weights[:,1]**2))).to(device)


# Armar mascara de generadores para la salida
idx_gen = net.gen["bus"].to_numpy()
idx_grid = net.ext_grid["bus"].to_numpy()
idx_gens = np.append(idx_gen,idx_grid,axis=0)
feature_mask = np.zeros(len(net.bus.index), dtype=int)
feature_mask[idx_gens] = 1
feature_mask = torch.Tensor(feature_mask).type(torch.int32).to(device)

/home/iboero/.local/lib/python3.11/site-packages/pandapower/io_utils.py:509: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_axes=False, **self.d)
/home/iboero/.local/lib/python3.11/site-packages/pandapower/io_utils.py:509: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_axes=False, **self.d)
/home/iboero/.local/lib/python3.11/site-packages/pandapower/io_utils.py:509: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_axes=False, **self.d)
/home/iboero/.local/li

## Levanatar Datos

In [18]:
# Levantar los datos
datos_path = './data'

if red == "30":
    X = np.load(datos_path+'/input30.npy')
    y = np.load(datos_path+'/p_opt30.npy')
else:
    X = np.load(datos_path+'/input118.npy')
    y = np.load(datos_path+'/p_opt118.npy')

# Separar en X e Y 
X_tensor = torch.Tensor(X).to(device)
y_tensor_local = torch.Tensor(y[:,:,0]).to(device)

wo_ext_grid = False
if wo_ext_grid:
    y_tensor_local[:,0] = 0

In [19]:
# Generar loaders para entrenamiento
batch_size = 64

dataset_local = TensorDataset(X_tensor, y_tensor_local)
data_train_local,data_test_local = train_test_split(dataset_local,test_size=0.2,random_state=42)
data_train_local,data_val_local = train_test_split(data_train_local,test_size=0.1,random_state=42)

train_loader_local = DataLoader(data_train_local, batch_size=batch_size)
val_loader_local = DataLoader(data_val_local, batch_size=batch_size)
test_loader_local = DataLoader(data_test_local, batch_size=batch_size)


## Entrenamiento


### Funciones antrenamiento y evaluacion

In [20]:
def train_local(model, train_loader, optimizer, criterion, feature_mask,epoch=None,writer=None):
    model.train()
    total_loss = 0
    total_metric = 0
    normalized_error = NormalizedError()

    for data in train_loader:
        optimizer.zero_grad()
        output, feature_mask = model(data[0])
        target = data[1]

        # Compute the MSE loss only for nodes with missing features
        target = torch.masked_select(target, feature_mask.unsqueeze(0).bool()).view(target.shape[0],torch.sum(feature_mask))
        output = torch.masked_select(output, feature_mask.unsqueeze(0).bool()).view(output.shape[0],torch.sum(feature_mask))
        loss = criterion(output, target)  # target should contain true values for nodes with missing features
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        # Calcular la métrica
        metric = normalized_error(output, target)
        total_metric += metric.item()

    avg_loss = total_loss / len(train_loader)
    avg_metric = total_metric / len(train_loader)

    writer.add_scalar('Train/Training Loss', avg_loss, epoch)
    writer.add_scalar('Train/Training Metric', avg_metric, epoch)
    
    return avg_loss, avg_metric

def evaluate_local(model, data_loader, criterion, feature_mask,epoch=None, writer=None,test=False):
    model.eval()
    total_loss = 0
    total_metric = 0
    normalized_error = NormalizedError()

    with torch.no_grad():
        for data in data_loader:
            output, feature_mask = model(data[0])
            target = data[1]

            # Compute the MSE loss only for nodes with missing features
            target = torch.masked_select(target, feature_mask.unsqueeze(0).bool()).view(target.shape[0],torch.sum(feature_mask))
            output = torch.masked_select(output, feature_mask.unsqueeze(0).bool()).view(output.shape[0],torch.sum(feature_mask))
            loss = criterion(output, target)  # target should contain true values for nodes with missing features
            total_loss += loss.item()
            metric = normalized_error(output, target)
            total_metric += metric.item()
            
    avg_loss = total_loss / len(data_loader)
    avg_metric = total_metric / len(data_loader)

    if epoch != None:
        writer.add_scalar('Val/Validation Loss', avg_loss, epoch)
        writer.add_scalar('Val/Validation Metric', avg_metric, epoch)
    
    return avg_loss, avg_metric


### GNN

In [26]:
%autoreload

writer_gnn = SummaryWriter('resultados/gnn_local')

# Hiperparametros
dim = [4,128,64,1]
num_layers = 3
K = [3,3,0]
num_epochs = 200
N = len(net.bus)

if red == "118":
    lr = 0.001
else:
    lr = 0.0002

# Definicion Modelo, Optimizer y Loss
model_local = GNN_Local(dim, edge_index, edge_weights,num_layers, K, feature_mask,N).to(device)
optimizer = torch.optim.Adam(model_local.parameters(), lr=lr)
criterion = nn.MSELoss()  # Change the loss function as needed


    
# Entrenamiento
for epoch in range(num_epochs):
    train_loss, train_metric = train_local(model_local, train_loader_local, optimizer, criterion,feature_mask,epoch,writer_gnn)
    val_loss, val_metric = evaluate_local(model_local, val_loader_local, criterion,feature_mask, epoch,writer_gnn)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Metric: {train_metric:.4f},  Val Loss: {val_loss:.4f}, Val Metric: {val_metric:.4f}")

# Evaluacion
test_loss, test_metric = evaluate_local(model_local, test_loader_local, criterion, feature_mask,None,writer_gnn)
print(f"Test Loss: {test_loss:.5f}, Test Metric: {test_metric:.5f}")

Epoch 1/200, Train Loss: 23849.5174, Train Metric: 0.9930,  Val Loss: 23406.6880, Val Metric: 0.9884
Epoch 2/200, Train Loss: 22892.3965, Train Metric: 0.9829,  Val Loss: 22292.9946, Val Metric: 0.9764
Epoch 3/200, Train Loss: 21615.0444, Train Metric: 0.9689,  Val Loss: 20881.5936, Val Metric: 0.9606
Epoch 4/200, Train Loss: 20078.0611, Train Metric: 0.9512,  Val Loss: 19247.8830, Val Metric: 0.9412
Epoch 5/200, Train Loss: 18349.6251, Train Metric: 0.9300,  Val Loss: 17454.5440, Val Metric: 0.9185
Epoch 6/200, Train Loss: 16480.2695, Train Metric: 0.9053,  Val Loss: 15542.7859, Val Metric: 0.8922
Epoch 7/200, Train Loss: 14535.2066, Train Metric: 0.8773,  Val Loss: 13576.0361, Val Metric: 0.8625
Epoch 8/200, Train Loss: 12592.6704, Train Metric: 0.8463,  Val Loss: 11661.8564, Val Metric: 0.8304
Epoch 9/200, Train Loss: 10723.4045, Train Metric: 0.8130,  Val Loss: 9843.3719, Val Metric: 0.7959
Epoch 10/200, Train Loss: 8979.0320, Train Metric: 0.7777,  Val Loss: 8188.6810, Val Metric:

## FCNN


In [27]:
writer_fcnn = SummaryWriter('resultados/fcnn_local')

# Hiperparametros
dim = [4,128,64,1]
num_layers = 3
K = [0,0,0]
num_epochs = 200
N = len(net.bus)
if red == "118":
    lr = 0.001
else:
    lr = 0.0002


# Definicion Modelo, Optimizer y Loss
model_local = GNN_Local(dim, edge_index,edge_weights, num_layers, K, feature_mask,N).to(device)
optimizer = torch.optim.Adam(model_local.parameters(), lr=0.00013)
criterion = nn.MSELoss()  # Change the loss function as needed



# Entrenamiento
for epoch in range(num_epochs):
    train_loss, train_metric = train_local(model_local, train_loader_local, optimizer, criterion,feature_mask,epoch,writer_fcnn)
    val_loss, val_metric = evaluate_local(model_local, val_loader_local, criterion,feature_mask, epoch,writer_fcnn)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Metric: {train_metric:.4f},  Val Loss: {val_loss:.4f}, Val Metric: {val_metric:.4f}")

# Evaluacion
test_loss, test_metric = evaluate_local(model_local, test_loader_local, criterion, feature_mask,None,writer_fcnn)
print(f"Test Loss: {test_loss:.5f}, Test Metric: {test_metric:.5f}")


Epoch 1/200, Train Loss: 24178.3180, Train Metric: 0.9964,  Val Loss: 24009.4474, Val Metric: 0.9947
Epoch 2/200, Train Loss: 23952.6927, Train Metric: 0.9941,  Val Loss: 23904.8436, Val Metric: 0.9936
Epoch 3/200, Train Loss: 23856.3645, Train Metric: 0.9931,  Val Loss: 23810.6780, Val Metric: 0.9926
Epoch 4/200, Train Loss: 23761.9605, Train Metric: 0.9921,  Val Loss: 23715.8848, Val Metric: 0.9916
Epoch 5/200, Train Loss: 23664.8235, Train Metric: 0.9911,  Val Loss: 23605.7596, Val Metric: 0.9905
Epoch 6/200, Train Loss: 23543.5353, Train Metric: 0.9898,  Val Loss: 23488.9572, Val Metric: 0.9892
Epoch 7/200, Train Loss: 23431.6919, Train Metric: 0.9887,  Val Loss: 23377.7150, Val Metric: 0.9881
Epoch 8/200, Train Loss: 23318.4126, Train Metric: 0.9875,  Val Loss: 23263.0493, Val Metric: 0.9868
Epoch 9/200, Train Loss: 23200.3271, Train Metric: 0.9862,  Val Loss: 23144.0176, Val Metric: 0.9856
Epoch 10/200, Train Loss: 23076.4796, Train Metric: 0.9849,  Val Loss: 23018.0219, Val Metr